<table>
    <tr>
      <td>Introducción a
      </td>
      <td>
      <img src="https://media.licdn.com/dms/image/D5612AQF7GSp3l4pztQ/article-cover_image-shrink_720_1280/0/1686548640655?e=1715817600&v=beta&t=WQzv1EMkEEwZ0QZ0PF1anRKIHCl5BBH_YPZHdDQsWPM"  width=150/>
      </td>
     </tr>
</table>
Rafa Caballero

# Escalado: estandarización y normalización

### Índice
[Ejemplo](#ejemplo)<br>
[Estandarización](#estandarización)<br>
[Normalización](#normalización)<br>
[Otros normalizadores](#otros)<br>

<a name="ejemplo"></a>
#### Ejemplo

Tenemos datos de altura y número de zapato de unas cuantas personas

In [ ]:
import statistics as s
import numpy as np
import pandas as pd

size = 100
n = s.NormalDist(mu=1.76, sigma=0.16) # generador de números siguiendo una normal N(1.76,0.16)
altura = n.samples(size,seed=5)
altura = np.array(altura).round(4)
n2 = s.NormalDist(mu=40, sigma=3.2) # generador de números siguiendo una normal N(40,3.2)
zapato = n2.samples(size,seed=30)
zapato = np.array(zapato).round(2)

df = pd.DataFrame({"alto":altura,"pie":zapato})

df.hist()


In [ ]:
altura

In [ ]:
zapato

Los datos no son nada "reales" (¿por qué?) pero nos valen para un experimento sencillo

In [ ]:
df.sample(10).sort_values(by=["alto"])

Supongamos ahora que tenemos una actividad que requiere tener pie pequeño y bastante altura, digamos 36 de pie y 1.95 de altura. ¿Qué valores en df se acercan más a este estándar? Podemos utilizar la fórmula de la distancia para ello:

$\sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2} = [(x_2 - x_1)^2 + (y_2 - y_1)^2]^{(\frac{1}{2})}$

In [ ]:
x1 = 1.95
y1 = 36

df2 = df.copy()
df2["dist"] = ((df.alto-x1)**2 + (df.pie-y1)**2)**0.5

df2.sort_values(by=["dist"])

¿Qué ocurre? ¿Por qué no funciona?

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(5, 3))
dfs = df
ax.scatter(dfs.alto,dfs.pie,s=0.5)
plt.xlim(0,50)
plt.ylim(0,50)
plt.xlabel("alto")
plt.ylabel("pie")
plt.show()

¡¡Prácticamente hay una sola dimensión!! Esto va a influir mucho en los métodos de naturaleza geométrica (logística, regresión linea, KNN, SVM, redes neuronales...) porque la altura no influirá, es como si estuviéramos diciendo que es mucho menos importante.

<a name="estandarizacion"></a>
#### Estandarización

¿Cuál es la solución? **Estandarizar**  (también conocida como *estandarización Z*, o *Z-score standarization*), que es la forma de escalado más habitual entre datos que siguen aproximadamente una normal (y que a veces también se usa con datos que no la siguen!)


La idea es convertir las distribuciones en una N(0,1) para facilitar la comparación

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*dZlwWGNhFco5bmpfwYyLCQ.png" width=300/>

La transformación es sencilla, dada una columna $x$ con media $\mu$ y desviación típica $\sigma$ vamos a construir una versión:


$$
z = \frac{x - \mu}{\sigma}
$$

Vamos a hacerlo primero "a mano"

In [ ]:
media_alto,media_pie = df.mean()
dt_alto,dt_pie = df.std()

dfs = df.copy()
dfs["alto_s"] = (df.alto-media_alto)/dt_alto
dfs["pie_s"] = (df.pie-media_pie)/dt_pie
dfs.describe()

In [ ]:
media_alto_s,media_pie_s = dfs[["alto_s","pie_s"]].mean()
dt_alto_s,dt_pie_s = dfs[["alto_s","pie_s"]].std()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(5, 3))
ax.scatter(dfs.alto_s,dfs.pie_s,s=4)
plt.xlim(-3,3)
plt.ylim(-3,3)
plt.xlabel("alto")
plt.ylabel("pie")
plt.show()

Ahora sí tienen importancia ambas variables

In [ ]:
dfs

In [ ]:
x1 = (1.95-media_alto)/dt_alto
y1 = (36-media_pie)/dt_pie

df2 = dfs.copy()
df2["dist"] = ((dfs.alto_s-x1)**2 + (dfs.pie_s-y1)**2)**0.5

df2.sort_values(by=["dist"])

Mostramos el valor de referencia y los más cercanos

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(5, 3))
ax.scatter(dfs.alto_s,dfs.pie_s,s=4,color="blue")
ax.scatter([x1],[y1],s=40,color="green")
cercanos = df2.sort_values(by=["dist"]).head(10)
ax.scatter(cercanos.alto_s,cercanos.pie_s,s=4,color="red")

plt.xlim(-3,3)
plt.ylim(-3,3)
plt.xlabel("alto")
plt.ylabel("pie")
plt.show()

In [ ]:
x1,y1

Hay muchos métodos que ML que requerirán estandarización, tantos que aunque es una trnasformación bastante simple de implementar es muy normal utilizar la versión incluída en la bibliteca sklearn

In [ ]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler() # 1 declarar el método
scaler.fit(df)  # 2 "aprender" de los datos (en este caso obtener la media y la desviación típica)
datos = scaler.transform(df) # aplicar el método ya instanciado a los datos
dfs = pd.DataFrame(datos,columns=["alto_s","pie_s"]) # convertir a DataFrame
dfs

In [ ]:
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(5, 3))
ax.scatter(dfs.alto_s,dfs.pie_s,s=4,color="blue")
ax.scatter([x1],[y1],s=40,color="green")
cercanos = df2.sort_values(by=["dist"]).head(10)
ax.scatter(cercanos.alto_s,cercanos.pie_s,s=4,color="red")

plt.xlim(-3,3)
plt.ylim(-3,3)
plt.xlabel("alto")
plt.ylabel("pie")
plt.show()

Un último apunte: el que queramos estandarizar (o usar otro escalado) va a depender de la situación. Por ejemplo siquieremos disitnguir días soleados de nublados a partir de datos de sensores puede ser una mala idea:

<img src="https://github.com/RafaelCaballero/tdm/raw/master/images/soleadonublado.png" />

<a name="normalizacion"></a>
#### Normalización

Hay otras formas de escalar alternativas, que en ocasiones resultan muy útiles, especialmente (pero no solo) con datos que no siguen una normal. El más habitual es el MIN-MAX scaler que convierte la columna al rango 0-1 aplicando esta sencilla transformación: dada una variable/columna $x$ con máximo $M$ y mínimo $m$, se define la transformación min-max $x'$

$$
x' = \frac{x-m}{M-m}
$$



In [ ]:
df

In [ ]:
alto_m, pie_m = df.min()
alto_M, pie_M = df.max()

In [ ]:
dfn = df.copy()
dfn["alto_n"] = (df.alto-alto_m)/(alto_M-alto_m)
dfn["pie_n"] = (df.pie-pie_m)/(pie_M-pie_m)
dfn

In [ ]:
xn = (1.95-alto_m)/(alto_M-alto_m)
yn = (36-pie_m)/(pie_M-pie_m)

df2n = dfn.copy()
df2n["dist"] = ((dfn.alto_n-xn)**2 + (dfn.pie_n-yn)**2)**0.5

df2n.sort_values(by=["dist"])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize=(5, 3))
ax.scatter(dfn.alto_n,dfn.pie_n,s=4,color="blue")
ax.scatter([xn],[yn],s=40,color="green")
cercanos = df2n.sort_values(by=["dist"]).head(10)
ax.scatter(cercanos.alto_n,cercanos.pie_n,s=4,color="red")

plt.xlim(0,1)
plt.ylim(0,1)
plt.xlabel("alto")
plt.ylabel("pie")
plt.show()

De nuevo podemos utilizar la librería

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler


scaler = MinMaxScaler() # 1 declarar el método
scaler.fit(df)  # 2 "aprender" de los datos (en este caso obtener la media y la desviación típica)
datos = scaler.transform(df) # aplicar el método ya instanciado a los datos
dfs = pd.DataFrame(datos,columns=["alto_s","pie_s"]) # convertir a DataFrame
dfs

In [ ]:
scaler.transform([[1.95,36]])

In [ ]:
xn,yn = scaler.transform([[1.95,36]])[0]

In [ ]:
df2n = dfs.copy()
df2n["dist"] = ((dfs.alto_s-xn)**2 + (dfs.pie_s-yn)**2)**0.5

df2n.sort_values(by=["dist"])

In [ ]:
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(5, 3))
ax.scatter(dfs.alto_s,dfs.pie_s,s=4,color="blue")
ax.scatter([xn],[yn],s=40,color="green")
cercanos = df2n.sort_values(by=["dist"]).head(10)
ax.scatter(cercanos.alto_s,cercanos.pie_s,s=4,color="red")

plt.xlim(0,1)
plt.ylim(0,1)
plt.xlabel("alto")
plt.ylabel("pie")
plt.show()

El usar el mismo esquema nos permite escribir una función que sea independiente del método

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

def escala(df,metodo,columnas):
    metodo.fit(df)
    datos = metodo.transform(df)
    return pd.DataFrame(datos,columns=columnas)


dfmM = escala(df, MinMaxScaler(), ["alto_s","pie_s"])
dfs  = escala(df, StandardScaler(), ["alto_s","pie_s"])



<a name="otros"></a>
#### Otros normalizadores
Un problema conocido en el caso de maxmin, y una razón por la que a menudo se prefiere la estandarización

In [ ]:
df2 = df.copy()
df2.iloc[0,0]=40 # un error en los datos
df2

In [ ]:
minmax =  MinMaxScaler()
dfs = escala(df2, minmax, ["alto_s","pie_s"])

# distancias
xn,yn = minmax.transform([[1.95,36]])[0]
df2n = dfs.copy()
df2n["dist"] = ((dfs.alto_s-xn)**2 + (dfs.pie_s-yn)**2)**0.5



fig, ax = plt.subplots(figsize=(5, 3))
ax.scatter(dfs.alto_s,dfs.pie_s,s=4,color="blue")
ax.scatter([xn],[yn],s=40,color="green")
cercanos = df2n.sort_values(by=["dist"]).head(10)
ax.scatter(cercanos.alto_s,cercanos.pie_s,s=4,color="red")

plt.xlim(0,0.1)
plt.ylim(0,1)
plt.xlabel("alto")
plt.ylabel("pie")
plt.show()

Para evitar este problema existen otras versiones como RobusScaler que se basan en la mediana

In [ ]:
from sklearn.preprocessing import RobustScaler

escalador =    RobustScaler()
dfs = escala(df2, escalador, ["alto_s","pie_s"])

# distancias
xn,yn = escalador.transform([[1.95,36]])[0]
df2n = dfs.copy()
df2n["dist"] = ((dfs.alto_s-xn)**2 + (dfs.pie_s-yn)**2)**0.5



fig, ax = plt.subplots(figsize=(5, 3))
ax.scatter(dfs.alto_s,dfs.pie_s,s=4,color="blue")
ax.scatter([xn],[yn],s=40,color="green")
cercanos = df2n.sort_values(by=["dist"]).head(10)
ax.scatter(cercanos.alto_s,cercanos.pie_s,s=4,color="red")

plt.xlim(-2,2)
plt.ylim(-2,2)
plt.xlabel("alto")
plt.ylabel("pie")
plt.show()

[Aquí](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html) se puede encontrar una comparativa del efecto que pueden tener los outliers comparando estos y otros escaladores